# Lesson 4：Python 迭代器详解

本节内容围绕“迭代器”展开：先从概念入手，再拆解底层协议，最后通过示例展示从手写迭代器到 `itertools` 的多种用法。

## 1. 基本概念：迭代器与可迭代对象

- **可迭代对象（Iterable）**：实现了 `__iter__()` 方法，或定义了适用于迭代的序列协议（如列表、字符串）。
- **迭代器（Iterator）**：在 `__iter__()` 之外还实现 `__next__()`，每次调用返回下一个元素。
- Python 内置函数 `iter()` 会从可迭代对象中获取迭代器，`next()` 则显式获取下一个值。

### 1.1 常见内置可迭代对象

- 序列类型：`list`、`tuple`、`range` 等都实现了 `__iter__`，每次遍历都会生成新的迭代器对象。
- 映射与集合：`dict` 默认迭代键，`set`、`frozenset` 等无序容器也提供迭代器。
- 文本与二进制流：字符串、字节串、文件对象都支持按元素或按行的迭代。
- 迭代器本身也是可迭代对象，但它们只能遍历一次，因为 `__next__` 会逐步耗尽内部状态。


In [2]:
# 检查常见内置对象的迭代能力
from collections.abc import Iterable, Iterator

samples = {
    "list": [1, 2, 3],
    "tuple": (1, 2, 3),
    "dict": {"a": 1, "b": 2},
    "set": {1, 2, 3},
    "str": "abc",
    "range": range(3),
}

for name, obj in samples.items():
    # iter() 会返回对象的迭代器；不同类型的返回值各不相同
    iterator = iter(obj)
    print(f"{name:>8} -> Iterable? {isinstance(obj, Iterable):5} | Iterator? {isinstance(obj, Iterator):5}")
    print(f"           iterator type: {type(iterator).__name__}")
    print(f"           has __next__? {'__next__' in dir(iterator)}")
    print('-' * 55)


    list -> Iterable?     1 | Iterator?     0
           iterator type: list_iterator
           has __next__? True
-------------------------------------------------------
   tuple -> Iterable?     1 | Iterator?     0
           iterator type: tuple_iterator
           has __next__? True
-------------------------------------------------------
    dict -> Iterable?     1 | Iterator?     0
           iterator type: dict_keyiterator
           has __next__? True
-------------------------------------------------------
     set -> Iterable?     1 | Iterator?     0
           iterator type: set_iterator
           has __next__? True
-------------------------------------------------------
     str -> Iterable?     1 | Iterator?     0
           iterator type: str_ascii_iterator
           has __next__? True
-------------------------------------------------------
   range -> Iterable?     1 | Iterator?     0
           iterator type: range_iterator
           has __next__? True
---------------

In [1]:
# 使用 iter() 与 next() 手动遍历列表，以观察迭代过程
numbers = [10, 20, 30]
iterator = iter(numbers)        # 通过 iter() 获取迭代器对象

print(next(iterator))           # 第一次调用 next() 返回 10
print(next(iterator))           # 第二次调用返回 20
print(next(iterator))           # 第三次调用返回 30

try:
    print(next(iterator))       # 再次调用触发 StopIteration 异常
except StopIteration:
    print("迭代完成，StopIteration 被捕获")


10
20
30
迭代完成，StopIteration 被捕获


## 2. 底层协议：`__iter__` 与 `__next__`

迭代器必须实现两个关键方法：

1. `__iter__(self)`：返回迭代器本身，一般写 `return self`，便于在 `for` 循环中重复调用。
2. `__next__(self)`：返回下一个值，若无更多元素则抛出 `StopIteration`。`for` 循环会捕获该异常并悄悄结束迭代。

`iter(obj)` 与 `next(it)` 只是对这两个魔术方法的友好封装。要识别某个对象是否符合协议，可以借助 `collections.abc.Iterable` 与 `Iterator` 这两个抽象基类。


### 2.1 CPython 底层实现概览

- `iter(obj)` 在 CPython 中调用 `PyObject_GetIter`，该函数会查找对象的 `__iter__` 方法或序列协议来创建迭代器。
- 内置容器会返回专门的 C 级别迭代器类型，例如 `list_iterator`、`dict_keyiterator`、`set_iterator`。
- 迭代器维护内部游标，每次 `__next__` 调用都会移动游标；到达末尾后抛出 `StopIteration`。
- `collections.abc` 提供统一的 ABC（抽象基类）来判断对象是否符合 Iterable/Iterator 接口，方便在运行时做类型检查。


In [3]:
# 利用抽象基类区分 Iterable 与 Iterator
from collections.abc import Iterable, Iterator

list_obj = [1, 2, 3]
list_iterator = iter(list_obj)
gen_expression = (n ** 2 for n in range(3))

objects = {
    "list_obj": list_obj,
    "list_iterator": list_iterator,
    "generator": gen_expression,
}

for name, obj in objects.items():
    print(f"{name:>14}: Iterable? {isinstance(obj, Iterable):5} | Iterator? {isinstance(obj, Iterator):5}")

# generator 本身就是迭代器；再次调用 iter() 会返回自身
print(iter(gen_expression) is gen_expression)


      list_obj: Iterable?     1 | Iterator?     0
 list_iterator: Iterable?     1 | Iterator?     1
     generator: Iterable?     1 | Iterator?     1
True


In [1]:
# 从零开始实现一个计算平方数的迭代器
class SquareIterator:
    def __init__(self, limit):
        # 保存迭代的上限，用于决定何时停止
        self.limit = limit
        # 记录下一次返回的基数，从 0 开始方便演示
        self.current = 0

    def __iter__(self):
        # 迭代器的 __iter__ 直接返回自身，方便在 for 循环中使用
        return self

    def __next__(self):
        # 当 current 超过 limit 时终止迭代
        if self.current > self.limit:
            raise StopIteration
        # 在返回前预先计算本次结果
        value = self.current ** 2
        # 更新状态，确保下一次调用返回下一个平方数
        self.current += 1
        return value

squares = SquareIterator(limit=3)
for number in squares:
    print(number)


0
1
4
9


## 3. 迭代器与自定义容器

如果构建的是容器类（例如包装列表的集合类型），通常通过在容器的 `__iter__()` 方法中返回内部数据结构的迭代器，实现“可迭代但非迭代器”的模式：

In [ ]:
# 自定义容器类，让其可迭代但本身不是迭代器
class TagCollection:
    def __init__(self, *tags):
        # 存放标签的底层列表
        self._tags = list(tags)

    def __iter__(self):
        # 返回列表自带的迭代器对象
        return iter(self._tags)

    def add(self, tag):
        # 示例方法：向集合中追加新标签
        self._tags.append(tag)

favorite_tags = TagCollection("python", "生成器", "itertools")
for tag in favorite_tags:
    print(tag)

# 再次遍历不会出问题，因为每次 for 循环都会调用 __iter__ 生成新的迭代器
for tag in favorite_tags:
    print(tag.upper())


## 4. 生成器：语法糖式的迭代器

生成器函数通过 `yield` 关键字自动实现迭代协议，写法更简洁：

In [4]:
# 用生成器函数输出斐波那契数列

def fibonacci(limit):
    # 初始化前两个基数
    a, b = 0, 1
    count = 0
    while count < limit:
        yield a                 # yield 会暂停函数并返回当前值
        a, b = b, a + b         # 更新两个变量以生成下一个数
        count += 1

for value in fibonacci(6):
    print(value)


0
1
1
2
3
5


## 5. 生成器表达式与惰性计算

除了生成器函数，Python 还提供生成器表达式，用圆括号包裹推导式：

In [ ]:
# 生成器表达式用于惰性生成数据，避免一次性占用大量内存
squared_numbers = (n ** 2 for n in range(5))
print(next(squared_numbers))    # 惰性求值，逐个生成
print(list(squared_numbers))    # 将剩余元素耗尽并收集为列表


## 6. `itertools`：构建复杂迭代流程

标准库 `itertools` 提供了大量高性能迭代器构件件，可以按乐高积木一样组合：

In [2]:
# 组合 itertools 工具快速构建复杂迭代逻辑
import itertools

# cycle 可以无限循环某个序列；islice 用于截取有限长度，防止无限循环
cycled = itertools.islice(itertools.cycle(["A", "B", "C"]), 7)
print(list(cycled))

# chain 可以把多个可迭代对象拼接在一起
combined = itertools.chain([1, 2], range(3, 5), (5, 6))
print(list(combined))

# accumulate 可以在迭代过程中累加，生成前缀和序列
prefix_sum = itertools.accumulate([1, 2, 3, 4])
print(list(prefix_sum))


['A', 'B', 'C', 'A', 'B', 'C', 'A']
[1, 2, 3, 4, 5, 6]
[1, 3, 6, 10]


## 7. 小结

- 迭代器依赖 `__iter__` 与 `__next__` 协议，`StopIteration` 用于终止。
- 自定义容器通常只需返回内部数据结构的迭代器。
- 生成器函数与表达式提供了直观的方式构建迭代器。
- 利用 `itertools` 可以高效地组合复杂的迭代模式。